In [25]:
!pip3 install torch_optimizer torchmetrics
!nvidia-smi

Tue Sep 20 10:19:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    32W / 250W |   4877MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_optimizer import Ranger
from torchvision import datasets
from torchvision import transforms,models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchmetrics import Accuracy
from torch.optim.lr_scheduler import CyclicLR

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Data pre-processing**

In [27]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=[32,32], padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.CenterCrop(size=[32,32]),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = datasets.CIFAR100(root='data/', download=True, transform=transform_train)
test_dataset = datasets.CIFAR100(root='data/', train=False, transform=transform_test)


val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
batch_size=128

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=16)
valid_loader = DataLoader(val_ds, batch_size, num_workers=16)
test_loader = DataLoader(test_dataset, batch_size, num_workers=16)

Files already downloaded and verified


In [28]:
num_classes = 100
num_epochs = 100

In [29]:
class VGGForCiFar100(nn.Module):
  def __init__(self, hid_dim=5000, dropout=0.5, n_classes=100, use_fc=True, freeze=False):
    super().__init__()
    self.vgg = models.vgg16(pretrained=True)
    if not use_fc:
      self.vgg.classifier = nn.Linear(512 * 7 * 7, n_classes)
    else:
      self.vgg.classifier = nn.Sequential(
              nn.Linear(512 * 7 * 7, 4096),
              nn.ReLU(True),
              nn.Dropout(p=dropout),
              nn.Linear(4096, 4096),
              nn.ReLU(True),
              nn.Dropout(p=dropout),
              nn.Linear(4096, n_classes),
          )
      
    if freeze:
      for param in self.vgg.features.parameters():
        param.requires_grad = False
      for param in self.vgg.avg_pool.parameters():
        param.requires_grad = False
      for param in self.vgg.flatten.parameters():
        param.requires_grad = False
    

  def forward(self, x):
    return F.log_softmax(self.vgg(x), dim=-1)

**VGG16 Architecture**

In [30]:
model = VGGForCiFar100(n_classes=num_classes).to(device)
criterion = nn.NLLLoss().to(device)
accuracy = Accuracy(num_classes=num_classes).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=1e-4) 
scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-3, step_size_up=len(train_loader)//2, cycle_momentum=False)

**Training and Validation**

In [31]:
total_train_step = len(train_loader)
total_val_step = len(valid_loader)
BEST_VAL_METRIC = 0
BEST_MODEL = None

for epoch in range(1, num_epochs+1):

    train_loss=0
    train_acc=0
    model.train()

    for i, (images, target) in enumerate(train_loader, 1):

        # Move tensors to the configured device
        images = images.to(device)
        target = target.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, target)

        train_loss += loss
        train_acc += accuracy(outputs, target)
        
        # Backward and optimize
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        
        optimizer.zero_grad()

    print(f'Epoch [{epoch}/{num_epochs}] - Loss: {(train_loss/total_train_step):.4f}, Accuracy: {(train_acc/total_train_step):.4f}')

    model.eval() 
    # Validation
    with torch.no_grad():
        val_acc = 0
        val_loss=0
        for i, (images, target) in enumerate(valid_loader, 1):
            images = images.to(device)
            target = target.to(device)

            outputs = model(images)
            val_loss += criterion(outputs, target)
            val_acc += accuracy(outputs, target)

    if val_acc/total_val_step > BEST_VAL_METRIC:
        BEST_VAL_METRIC = val_acc/total_val_step
        BEST_MODEL = model.state_dict() 

    print(f'Accuracy of the network on the 5000 validation images: {(val_acc/total_val_step):.4f}, loss: {(val_loss/total_val_step):.4f}') 

Epoch [1/100] - Loss: 4.5561, Accuracy: 0.0265
Accuracy of the network on the 5000 validation images: 0.0992, loss: 4.4191
Epoch [2/100] - Loss: 4.2060, Accuracy: 0.0835
Accuracy of the network on the 5000 validation images: 0.1844, loss: 3.6203
Epoch [3/100] - Loss: 3.5674, Accuracy: 0.1534
Accuracy of the network on the 5000 validation images: 0.2893, loss: 2.9103
Epoch [4/100] - Loss: 3.1182, Accuracy: 0.2195
Accuracy of the network on the 5000 validation images: 0.3320, loss: 2.5856
Epoch [5/100] - Loss: 2.8445, Accuracy: 0.2703
Accuracy of the network on the 5000 validation images: 0.3822, loss: 2.3746
Epoch [6/100] - Loss: 2.6528, Accuracy: 0.3085
Accuracy of the network on the 5000 validation images: 0.4033, loss: 2.2485
Epoch [7/100] - Loss: 2.5088, Accuracy: 0.3371
Accuracy of the network on the 5000 validation images: 0.4131, loss: 2.1417
Epoch [8/100] - Loss: 2.3969, Accuracy: 0.3590
Accuracy of the network on the 5000 validation images: 0.4426, loss: 2.0721
Epoch [9/100] - 

**Testing**

In [32]:
model.load_state_dict(BEST_MODEL)
total_test_step = len(test_loader)

model.eval()
with torch.no_grad():
    test_acc=0
    test_loss=0

    for i, (images, target) in enumerate(test_loader, 1):

        # Move tensors to the configured device
        images = images.to(device)
        target = target.to(device)

        # Forward pass
        outputs = model(images)
        
        # Loss
        test_loss += criterion(outputs,target)
        test_acc += accuracy(outputs, target)

    print(f'Accuracy of the network on test images: {(test_acc/total_test_step):.4f}, loss: {(test_loss/total_test_step):.4f}')

Accuracy of the network on test images: 0.6479, loss: 1.2986
